# **Thành viên NHÓM 1:**

1. Dương Thị Diễm Hương - 050608200211
2. Đỗ Thị Hồng Nhung - 050608200537
3. Đinh Thị Như Quỳnh - 030336200224








# SET UP

In [28]:
%%capture
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

<IPython.core.display.Javascript object>

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
     

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# Avoids scroll-in-the-scroll in the entire Notebook
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 400})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

Mounted at /content/drive


In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

<IPython.core.display.Javascript object>

--2023-04-02 16:29:00--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  19.5MB/s    in 0.7s    

2023-04-02 16:29:01 (19.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
%matplotlib inline
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
# create the session
conf = SparkConf().set("spark.ui.port", "4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

<IPython.core.display.Javascript object>




# QUESTION 1: SPARK

In [6]:
def line_to_friend_ownership(line):
    split = line.split()
    user_id = int(split[0])
    if len(split) == 1:
        friends = []
    else:
        friends = list(map(lambda x: int(x), split[1].split(',')))
    return user_id, friends

def friend_ownership_to_connection(f_o):
    user_id = f_o[0]
    friends = f_o[1]
    connections = []
    for friend_id in friends:
        key = (user_id, friend_id)
        if user_id > friend_id:
            key = (friend_id, user_id)
        connections.append((key, 0))  # they are friends, value=0
    for friend_pair in itertools.combinations(friends, 2):
        friend_0 = friend_pair[0]
        friend_1 = friend_pair[1]
        key = (friend_0, friend_1)
        if friend_0 > friend_1:
            key = (friend_1, friend_0)
        connections.append((key, 1))  # they have mutual friends, value=1
    return connections

def mutual_friend_count_to_recommendation(f):
    pair = f[0]
    friend0 = pair[0]
    friend1 = pair[1]
    noMutFriends = f[1]
    rec0 = (friend0, (friend1, noMutFriends))
    rec1 = (friend1, (friend0, noMutFriends))
    return [rec0, rec1]

def recommendation_to_sorted_truncated(recs):
    recs.sort(key=lambda x: (-x[1], x[0]))
    return list(map(lambda x: x[0], recs))[:10]
     

<IPython.core.display.Javascript object>

In [7]:
# Read from text file 
lines = sc.textFile("/content/drive/MyDrive/soc-LiveJournal1Adj.txt")

# Map each line to the form: (user_id, [friend_id_0, friend_id_1, ...])
friend_ownership = lines.map(line_to_friend_ownership).filter(lambda friend: '' != friend[1])#.filter(lambda friend: 1000> friend[0]) #take 1000 samples for testing

# Map friend ownerships to instances of ((user_id, friend_id), VALUE).
# VALUE = 0 => pairs are already friends.
# VALUE = 1 => pairs have mutual friends.
friend_edges = friend_ownership.flatMap(friend_ownership_to_connection)
friend_edges.cache()

# Filter pairs that are already friends
mutual_friend = friend_edges.groupByKey() \
    .filter(lambda edge: 0 not in edge[1]) \
    .flatMap(lambda x: [(x[0],item) for item in x[1]]) # flat it to count total mutual friends No; use map directly causes bugs

# Count mutual friends by adding up values
mutual_friend_counts = mutual_friend.reduceByKey( lambda x,y : x+y)

# Create the recommendation objects, group them by key, then sort and 
recommendations = mutual_friend_counts.flatMap(mutual_friend_count_to_recommendation).groupByKey() 

# Truncate the recommendations to the 10 most highly recommended.
recommendations10 = recommendations.map(lambda m: (m[0], recommendation_to_sorted_truncated(list(m[1])))).sortByKey() 

# Include in your writeup the recommendations for the users with following user IDs: 924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993.
results = recommendations10.filter(lambda recommendations: recommendations[0] in [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993])


<IPython.core.display.Javascript object>

In [8]:
    results.collect()

<IPython.core.display.Javascript object>

[(924, [439, 2409, 6995, 11860, 15416, 43748, 45881]),
 (8941, [8943, 8944, 8940]),
 (8942, [8939, 8940, 8943, 8944]),
 (9019, [9022, 317, 9023]),
 (9020, [9021, 9016, 9017, 9022, 317, 9023]),
 (9021, [9020, 9016, 9017, 9022, 317, 9023]),
 (9022, [9019, 9020, 9021, 317, 9016, 9017, 9023]),
 (9990, [13134, 13478, 13877, 34299, 34485, 34642, 37941]),
 (9992, [9987, 9989, 35667, 9991]),
 (9993, [9991, 13134, 13478, 13877, 34299, 34485, 34642, 37941])]

# QUESTION 2: ASSOCIATION RULES

 **(a)** 

Nhược điểm của việc sử dụng chỉ tiêu độ tin cậy (confidence) là nó không xem xét xác suất cơ sở của việc B xảy ra khi không có A. Điều này có thể gây vấn đề bởi vì giá trị độ tin cậy cao có thể được đạt được ngay cả đối với các tập hợp phần tử có tần suất thấp trong tập dữ liệu. Ví dụ, nếu phần tử A xảy ra hiếm khi, và phần tử B xảy ra với A chỉ do sự tình cờ, thì conf(A → B) có thể rất cao, dẫn đến thuật toán tin tưởng rằng mối liên hệ là mạnh mẽ khi thực tế là yếu.


Tuy nhiên, chỉ tiêu tăng trưởng (lift) và chỉ số thuyết phục (conviction) không gặp nhược điểm này vì chúng tính đến tần suất tổng thể của B. Chỉ số tăng trưởng tính đến tần suất của B bằng cách đo lường sự khác biệt so với sự độc lập thống kê, trong khi chỉ số thuyết phục đo lường tần suất A xuất hiện mà không có B bằng cách so sánh nó với tần suất mong đợi được tính dựa trên tần suất tổng thể của B. Cả hai chỉ tiêu này cung cấp một đánh giá chính xác hơn về sức mạnh của mối liên hệ giữa các tập hợp phần tử hơn chỉ tiêu độ tin cậy đơn lẻ.

**(b)**

Trong các chỉ tiêu được giới thiệu ở đây, chỉ số tăng trưởng (lift) là không đối xứng.

Để chứng minh rằng độ tin cậy (confidence) và chỉ số thuyết phục (conviction) là các chỉ tiêu đối xứng:

Độ tin cậy:

*   conf(A → B) = Pr(B|A) = Pr(A|B) * Pr(B) / Pr(A) (định lý Bayes)
*   conf(B → A) = Pr(A|B) = Pr(B|A) * Pr(A) / Pr(B)
*   Do đó, conf(A → B) = conf(B → A)

Chỉ cố thuyết phục:

*   conv(A → B) = 1 − S(B) / [1 − conf(A → B)] = [(1 - S(A ∪ B)) / S(A)] / [(1 - S(B)) / S(A ∪ B)]
*   conv(B → A) = 1 − S(A) / [1 − conf(B → A)] = [(1 - S(A ∪ B)) / S(B)] / [(1 - S(A)) / S(A ∪ B)]
*   Do đó, conv(A → B) = conv(B → A)


Để chứng minh rằng chỉ số tăng trưởng không đối xứng, chúng ta có thể sử dụng một ví dụ đơn giản:

*   Giả sử có 50 giao dịch trong số 100 chứa phần tử A và 60 giao dịch trong số 100 chứa phần tử B.
*   giả định rằng 40 giao dịch chứa cả A và B.
*   Khi đó, lift(A → B) = conf(A → B) / S(B) = 0.67 / 0.6 = 1.11
*   Tuy nhiên, lift(B → A) = conf(B → A) / S(A) = 0.67 / 0.5 = 1.33
*   Vì 1.11 không bằng 1.33, chúng ta thấy rằng lift không đối xứng.


**(c)**

Chỉ tiêu tăng trưởng (lift) và chỉ số thuyết phục (conviction) đều có tính chất này vì cả hai chỉ tiêu đạt giá trị tối đa của chúng cho tất cả các liên hệ hoàn hảo (perfect implications).


Ví dụ, giả sử chúng ta có luật hoàn hảo A → B và chúng ta tính toán các chỉ tiêu tương ứng. Nếu conf(A → B) hoặc conv(A → B) đạt giá trị tối đa của chúng (là 1), thì chúng ta nói là chỉ tiêu là mong muốn. Vì lift không đạt được giá trị tối đa đối với các liên hệ hoàn hảo (vì S(B) có thể bằng 0), nên nó không đáp ứng tính chất này.


Ứng dụng trong đề xuất sản phẩm: Từ khóa khuyến nghị sản phẩm là một trong những ví dụ về việc bán hàng chéo (cross-selling) thường được sử dụng bởi các nhà bán lẻ trực tuyến. Một phương pháp đơn giản để đưa ra đề xuất sản phẩm là đề xuất các sản phẩm đã được duyệt cùng nhau thường xuyên bởi khách hàng.


Để tìm ra các sản phẩm được duyệt cùng nhau thường xuyên, chúng ta có thể sử dụng thuật toán A-priori. Chương trình tìm kiếm sản phẩm cùng được trình bày trong đề bài có thể được lập trình bằng ngôn ngữ lập trình của bạn. 


Để tìm các sản phẩm đáng kể, chúng ta sẽ đặt giá trị hỗ trợ (support) bằng 100, và tìm các bộ mặt hàng có kích thước là 2 và 3. Chú ý rằng một vài hàng chứa các sản phẩm trùng lặp, nhưng chúng ta có thể bỏ qua các sản phẩm trùng lặp này.



# **2 (d)**

In [9]:
# Importing libraries
from collections import defaultdict
from itertools import combinations
from operator import itemgetter

<IPython.core.display.Javascript object>

In [10]:
# Load dataset and remove duplicates
data = []
with open('/content/drive/MyDrive/browsing.txt', 'r') as f:
    for line in f:
        items = set(line.strip().split())
        data.append(list(items))

<IPython.core.display.Javascript object>

In [11]:
# First pass to find frequent items
min_support = 100 # define minimum support
item_counts = defaultdict(int)
for basket in data:
    for item in basket:
        item_counts[item] += 1
frequent_items = set(item for item, count in item_counts.items() if count >= min_support)

<IPython.core.display.Javascript object>

In [12]:
# Second pass to find frequent itemsets of size 2
itemset_counts = defaultdict(int)
for basket in data:
    for itemset in combinations(basket, 2):
        if set(itemset).issubset(frequent_items):
            itemset_counts[itemset] += 1
frequent_itemsets = set(itemset for itemset, count in itemset_counts.items() if count >= min_support)

<IPython.core.display.Javascript object>

In [13]:
# Generate association rules for pairs of items
rules = []
for itemset in frequent_itemsets:
    for item in itemset:
        antecedent = frozenset([item])
        consequent = frozenset(frozenset(itemset) - antecedent)
        confidence = itemset_counts[itemset] / item_counts[item]
        rules.append((antecedent, consequent, confidence))

<IPython.core.display.Javascript object>

In [14]:
# Sort rules by confidence and print top 5
top_rules = sorted(rules, key=lambda x: (-x[2], tuple(x[0])))
for antecedent, consequent, confidence in top_rules[:5]:
    print(f"{tuple(antecedent)} -> {tuple(consequent)} : {confidence}")


<IPython.core.display.Javascript object>

('GRO38636',) -> ('FRO40251',) : 0.9906542056074766
('FRO92469',) -> ('FRO40251',) : 0.983510011778563
('DAI23334',) -> ('DAI62779',) : 0.9545454545454546
('DAI43868',) -> ('SNA82528',) : 0.8006756756756757
('DAI93865',) -> ('FRO40251',) : 0.7788461538461539


# **2 (e)**

In [15]:
# Load dataset and remove duplicates
data = []
with open('/content/drive/MyDrive/browsing.txt', 'r') as f:
    for line in f:
        items = set(line.strip().split())
        data.append(list(items))

<IPython.core.display.Javascript object>

In [16]:
# First pass to find frequent items
min_support = 100
item_counts = defaultdict(int)
for basket in data:
    for item in basket:
        item_counts[item] += 1
frequent_items = set(item for item, count in item_counts.items() if count >= min_support)

<IPython.core.display.Javascript object>

In [17]:
# Second pass to find frequent itemsets of size 3
itemset_counts = defaultdict(int)
for basket in data:
    for itemset in combinations(basket, 3):
        if set(itemset).issubset(frequent_items):
            itemset_counts[itemset] += 1
frequent_itemsets = set(itemset for itemset, count in itemset_counts.items() if count >= min_support)

<IPython.core.display.Javascript object>

In [18]:
# Generate association rules for pairs of items
rules = []
for itemset in frequent_itemsets:
    for item in itemset:
        antecedent = frozenset([item])
        consequent = frozenset(frozenset(itemset) - antecedent)
        confidence = itemset_counts[itemset] / item_counts[item]
        rules.append((antecedent, consequent, confidence))

<IPython.core.display.Javascript object>

In [19]:
# Sort rules by confidence and print top 5
top_rules = sorted(rules, key=lambda x: (-x[2], tuple(x[0])))
for antecedent, consequent, confidence in top_rules[:5]:
    print(f"{tuple(antecedent)} -> {tuple(consequent)} : {confidence}")

<IPython.core.display.Javascript object>

('FRO92469',) -> ('FRO40251', 'SNA80324') : 0.4040047114252061
('SNA18336',) -> ('ELE92920', 'DAI62779') : 0.34132581100141046
('ELE20847',) -> ('FRO40251', 'SNA80324') : 0.28361858190709044
('SNA18336',) -> ('ELE92920', 'DAI62779') : 0.2679830747531735
('SNA59903',) -> ('ELE17451', 'DAI62779') : 0.22671156004489337


# QUESTION 3: LOCALITY-SENSITIVE HASHING

**(a)**

Giả sử có một cột chứa m giá trị 1 và do đó cột này còn lại có n - m giá trị 0. Chúng ta chọn ngẫu nhiên k hàng để tính toán minhash cho cột này. Ta cần chứng minh rằng xác suất để thu được giá trị "không biết" (tức là minhash đối với cột này là "không biết") là không quá ((n - k) / n) ^ m.

Để tính xác suất đó, ta cần tính xác suất để không có hàng nào trong các k hàng chúng ta chọn có giá trị 1 trong cột này.

Xác suất để lấy được một hàng có giá trị 1 trong cột này là m/n, vì có m giá trị 1 trong n giá trị tổng thể. Do đó, xác suất để một hàng không có giá trị 1 trong cột này là (n - m) / n.

Vì chúng ta đã chọn ra k hàng để xét, xác suất để tất cả k hàng đều không có giá trị 1 trong cột này là ((n - m) / n) ^ k. Và vì thuật toán minhash trả về giá trị "không biết" nếu tất cả k hàng đều có giá trị 0 trong cột này, xác suất của việc này chính là ((n-k)/n)^m.

Kết hợp lại, ta thu được kết luận rằng xác suất để thu được giá trị "không biết" đó là không quá ((n-k)/n)^m.

**(b)**

Chúng ta muốn xác suất của "không biết" không quá e^(-10), điều đó có nghĩa:

((n-k) / n) ^ m ≤ e^(-10)

Lấy logarithm tự nhiên 2 vế, ta có:

m * ln((n-k) / n) ≤ -10

ln((n-k) / n) ≤ -10 / m

(n-k) / n ≤ e^(-10 / m)

n - k ≤ n * e^(-10 / m)

k ≥ n * (1 - e^(-10 / m))

Vì n lớn hơn rất nhiều so với m hoặc k, nên ta có thể giả sử e^(-10 / m) là một lượng rất nhỏ. Sử dụng gợi ý từ câu hỏi, ta có thể xấp xỉ (1 - e^(-10/m)) bằng 1/e. Vì vậy:

k ≥ n / e

Vì vậy, giá trị nhỏ nhất của k để đảm bảo xác suất của "không biết" không quá e^(-10) là khoảng n / e.

**(c)**

***a)*** Xét ma trận sau đây với hai cột:

        | A | B |
    S1: | 1 | 1 |
    S2: | 0 | 1 |
    S3: | 1 | 0 |

***b)*** Tương đồng Jaccard của S1 và S2:

*   S1 giao S2 = {1}, S1 hợp S2 = {0, 1}.
*   Vì vậy Jaccard tương đồng = |S1 giao S2| / |S1 hợp S2| = 1/2.

***c)***  Xác suất của một chuỗi hoán vị vòng tròn ngẫu nhiên cho giá trị minhash giống nhau cho cả S1 và S2:

*   Chỉ có 3 hoán vị vòng tròn có thể: (S1 S2 S3), (S2 S3 S1), và (S3 S1 S2).
*   Với mỗi hoán vị cuộn vòng, ta tính chữ ký minhash của S1 và S2 bằng cách bắt đầu từ một hàng được chọn ngẫu nhiên và theo thứ tự hoán vị cuộn vòng.
*   Chữ ký minhash kết quả cho S1 và S2 như sau:

Permutation 1: (S1 S2 S3)

minhash(S1) = 1-0-1

minhash(S2) = 1-1-0

Permutation 2: (S2 S3 S1)

minhash(S1) = 1-1-1

minhash(S2) = 0-1-1

Permutation 3: (S3 S1 S2)

minhash(S1) = 1-1-0

minhash(S2) = 0-0-1

*   Ta thấy chỉ có trong hoán vị 1, giá trị minhash cho S1 và S2 giống nhau (tức là 1-0-1 và 1-1-0). Do đó, xác suất của một chuỗi hoán vị vòng tròn ngẫu nhiên cho giá trị minhash giống nhau cho cả S1 và S2 là 1/3, không bằng với tương đồng Jaccard là 1/2.
*   Ví dụ này cho thấy các hoán vị vòng tròn không đủ để ước lượng tương đồng Jaccard một cách chính xác.


# QUESTION 4: LSH FOR APPROXIMATE NEAR NEIGHBOR SEARCH 

**(a)**

Để chứng minh bất đẳng thức trên, chúng ta sử dụng định lý Chernoff.

Giả sử X là biến ngẫu nhiên đếm số lượng điểm dữ liệu trong WJ (1 ≤ J ≤ L) có khoảng cách lớn hơn cλ so với điểm truy vấn z. Sau đó X có phân phối nhị thức với các thông số n = |WJ| và p = Pr[d(x, z) > cλ].

Theo định nghĩa của T, chúng ta có |T ∩ WJ| ≤ X cho tất cả 1 ≤ J ≤ L. Do đó, ta có:

*   |T ∩ WJ| ≤ X ⇒ ∑_{J=1}^L|T ∩ WJ| ≤ ∑_{J=1}^LX

Chúng ta muốn tìm một giới hạn trên cho xác suất tổng các biến ngẫu nhiên X1, X2, ..., XL vượt quá một ngưỡng nào đó. Sử dụng định lý Chernoff, ta có:


*   Pr[ ∑_{J=1}^L XJ ≥ 3L ] ≤ exp(-λL) * E[e^(λ ∑_{J=1}^L XJ)]

trong đó λ > 0 là một tham số sẽ được chọn sau.

Bây giờ chúng ta cần tính hàm sinh moment (mgf) của XJ. Với phân phối nhị thức có thông số n và p, hàm sinh moment được cho bởi:

*   M_X(t) = E[e^(tX)] = (1-p + pe^t)^n

Trong trường hợp của chúng ta, ta có n = |WJ| và p = Pr[d(x, z) > cλ]. Vì vậy, chúng ta có:

*   M_X(t) = E[e^(tX)] = (1-p + pe^t)^{|WJ|}

Sử dụng biểu thức trên, ta có thể tính được hàm sinh moment của ∑_{J=1}^L XJ:

*   M_Y(t) = E[e^(t ∑_{J=1}^L XJ)] = ∏_{J=1}^L E[e^(tXJ)] = ∏_{J=1}^L M_X(t)

Bây giờ ta có thể sử dụng hàm sinh moment này trong bất đẳng thức Chernoff:

Pr[ ∑_{J=1}^L XJ ≥ 3L ] ≤ exp(-λL) * E[e^(λ ∑_{J=1}^L XJ)]

≤ exp(-λL) * M_Y(λ)

≤ exp(-λL) * ∏_{J=1}^L M_X(λ)


Lấy logarithm tự nhiên của cả hai giới hạn và tiếp tục đơn giản hóa, ta có:

*   ln Pr[ ∑_{J=1}^L XJ ≥ 3L ] ≤ -λL + ∑_{J=1}^L ln M_X(λ)

Chúng ta cần chọn λ để giới hạn trên đạt giá trị nhỏ nhất. Để làm được điều này, chúng ta lấy đạo hàm của giới hạn bên phải theo λ, và cho nó bằng không:

*   d/dλ ( -λL + ∑_{J=1}^L ln M_X(λ) ) = -L + ∑_{J=1}^L d/dλ ln M_X(λ) = 0

Giải phương trình trên cho λ, ta có:

*   λ = (1/L) ∑_{J=1}^L ln(1/pJ)

Thay giá trị λ này vào giới hạn, ta có:

*   Pr[ ∑_{J=1}^L XJ ≥ 3L ] ≤ exp(-∑_{J=1}^L ln(1/pJ) + ln(3))

Sau khi đơn giản hóa, ta có:

*   Pr[ ∑_{J=1}^L XJ ≥ 3L ] ≤ 1/3 * ∏_{J=1}^L (1/pJ)^{pJ} * 3

Vì log(1/x) ≥ (1-x) với 0 < x < 1, ta có:

*   ∑_{J=1}^L ln(1/pJ) = ∑_{J=1}^L log(1/pJ)
*   ≥ ∑_{J=1}^L (1-pJ)
*   = L - ∑_{J=1}^L pJ

Thay vào bất đẳng thức trên, ta có:

*   Pr[ ∑_{J=1}^L XJ ≥ 3L ] ≤ 1/3 * ∏_{J=1}^L (1/pJ)^{pJ} * 3
*   ≤ 1/3 * exp(-L + ∑_{J=1}^L pJ) * 3
*   ≤ 1/3

Do đó, chúng ta đã chứng minh được rằng:

*   Pr[∑_{J=1}^L |T ∩ WJ| ≥ 3L] ≤ 1/3

**(b)**
Vì d(x∗, z) ≤ λ, cho nên đối với bất kỳ 1 ≤ j ≤ L, chúng ta có Pr [gj(x∗) = gj(z)] ≥ pk1, và do đó Pr [gj(x∗) ≠ gj(z)] ≤ 1 − pk1 = 1 - 1/L, trong đó phương trình cuối cùng được tính bằng pk1 = plog1/p2(n)1 = n−log(1/p1)log(1/p2) = n−ρ = L−1.

Sau đó, do tính độc lập của các gj, ta có: Pr [∀1≤j≤L, gj(x∗) ≠ gj(z)] ≤ (1 - 1/L)L ≤ 1/e.


**(c)**

Từ hai bất đẳng thức, Pr[∑_{j=1}^L |T∩Wj| ≥ 3L] ≤ 1/3 và Pr[∀1≤j≤L, gj(x*)≠gj(z)] < 1/e, ta có thể kết luận rằng với xác suất không nhỏ hơn 2/3e, tức là một hằng số cố định, điểm được báo cáo x * là một (c, λ)-ANN thực sự của z trong T.

Để chứng minh điều này, giả sử rằng điểm được báo cáo x * không phải là một (c, λ)-ANN thực sự của z trong T. Theo định nghĩa của (c,λ)-ANN, ta có:

*   Pr[d(x*,z)>cλ] < 1/L

Áp dụng bất đẳng thức Markov cho biểu thức này, ta có:

*   Pr[d(x*,z)>cλ] < E[d(x*,z)]/(cλ) = 1/(cλ)

Vì vậy:

*   Pr[d(x*,z)≤cλ] ≥ 1 - 1/(cλ) ≥ 1 - cλ

Do đó, ta có:

*   Pr[∀1≤j≤L, gj(x*)≠gj(z)]
*   ≤ ∑_{j=1}^L Pr[|fj(x*)-fj(z)|>0]
*   ≤ ∑_{j=1}^L Pr[d(x*,z)>cλ]
*   ≤ LPr[d(x*,z)>cλ]
*   < 1

Điều này dẫn đến mâu thuẫn với bất đẳng thức Pr[∀1≤j≤L, gj(x*)≠gj(z)] < 1/e. Vì vậy, giả định ban đầu là sai và với xác suất không nhỏ hơn 2/3e, điểm được báo cáo x * là một (c, λ)-ANN thực sự của z trong T, tức là điểm mà có khoảng cách đến z không vượt quá cλ.

# **4 (d)**

In [39]:
!pip install lsh

<IPython.core.display.Javascript object>

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
from lsh import *
import time
from tabulate import tabulate
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [46]:
# Load the dataset
A = np.loadtxt('/content/drive/MyDrive/patches.csv', delimiter=',')

<IPython.core.display.Javascript object>

In [47]:
# Defining the indices of the patches we want to search
query_indices = range(100, 1100, 100)
print("Query points: {}".format([i for i in query_indices]))

# Define LSH parameters
L = 10
k = 24
num_neighbors = 3


<IPython.core.display.Javascript object>

Query points: [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]


In [ ]:
# Run LSH setup
functions, hashed_A = lsh_setup(A, k=k, L=L)

In [ ]:
# Perform LSH and linear search
lsh_search_times = [] # Time taken to run LSH search
linear_search_times = [] # Time taken to run linear search
lsh_neighbors_list = [] # List of LSH neighbors
linear_neighbors_list = [] # List of linear neighbors

for query_index in query_indices:
    # Run LSH search
    start_time = time.time()
    lsh_neighbors = lsh_search(A, hashed_A, functions, query_index, num_neighbors=num_neighbors)
    lsh_neighbors_list.append(lsh_neighbors)
    lsh_search_times.append(time.time() - start_time)
    
    # Run linear search
    start_time = time.time()
    linear_neighbors = linear_search(A, query_index, num_neighbors=num_neighbors)
    linear_neighbors_list.append(linear_neighbors)
    linear_search_times.append(time.time() - start_time)

In [ ]:
# Print results
print("LSH Average Search Time: {:.4f} seconds".format(np.mean(lsh_search_times)))
print("Linear Average Search Time: {:.4f} seconds".format(np.mean(linear_search_times)))

headers = ['Query Index', 'LSH Neighbors', 'Linear Neighbors']
table = []
for i in range(len(query_indices)):
    table.append([query_indices[i], 
                  ", ".join(map(str, lsh_neighbors_list[i])), 
                  ", ".join(map(str, linear_neighbors_list[i]))])

print("Top 3 Neighbors for Each Query Point:")
print(tabulate(table, headers=headers, tablefmt='fancy_grid'))